---
title: "Homework 1"
author: "Paul Trusela"
format:
  html:
    toc: true
    toc-location: left
    self-contained: true
jupyter: python3
---


Professional wrestling, while not everyone's cup of tea, is big business. What started as a carnival act has turned into a global entertainment industry. Netflix recently started showing Monday Night Raw, a program from the biggest North American wrestling company, WWE -- this deal is reportedly worth \$5 billion. Like any large entity, WWE is not without competition, drama, and scandal. 

## General Tips

This is very much a step-by-step process. Don't go crazy trying to get everything done with as few lines as possible. Read the documentation for the AlphaVantage api! Carefully explore the pages from cagematch. There isn't a need to get too fancy with anything here -- just go with simple function and all should be good. Don't print comments, but use normal text for explanations.

## Step 1

In the `calls` folder, you'll find 4 text files -- these are transcripts from quarterly earnings calls. Read those files in (glob.glob will be very helpful here), with appropriate column names for ticker, quarter, and year columns; this should be done within a single function. Perform any data cleaning that you find necessary. 


In [ ]:
import glob as glob
import pandas as pd

glob.glob("G:/My Drive/MSSA60250 Unstructured Seth/unstructured_notes/calls/*")

def read_calls(folder_path):
    files = glob.glob(folder_path + "/*.txt")
    data = []
    for file in files:
        file_name = file.split("/")[-1]  
        parts = file_name.split("_")
        ticker = parts[0]
        quarter = parts[1].replace("Q", "")
        year = parts[2].replace(".txt", "")
        with open(file, "r", encoding="utf-8") as f:
            content = f.read()
        data.append({"ticker": ticker, "quarter": quarter, "year": year, "content": content})
    df = pd.DataFrame(data)
    
    return df

folder_path = "G:/My Drive/MSSA60250 Unstructured Seth/unstructured_notes/calls"
calls_df = read_calls(folder_path)

calls_df['ticker'] = calls_df['ticker'].str.replace('calls\\', '', regex=False)

print(calls_df.head())

## Step 2

Use the AlphaVantage api to get daily stock prices for WWE and related tickers for the last 5 years -- pay attention to your data. You cannot use any AlphaVantage packages (i.e., you can only use requests to grab the data). Tell me about the general trend that you are seeing. I don't care which viz package you use, but plotly is solid and plotnine is good for ggplot2 users.


In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"
import requests
import pandas as pd
import plotly.express as px
from datetime import datetime

def fetch_stock_data(symbol, api_key):
    """
    Fetch daily stock price data using TIME_SERIES_DAILY (free-tier Alpha Vantage).
    Args:
        symbol (str): Stock ticker symbol.
        api_key (str): AlphaVantage API key.
    Returns:
        pd.DataFrame: DataFrame with daily stock prices.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",  
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": "full"  
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "Time Series (Daily)" not in data:
        print(f"Error fetching data for {symbol}: {data.get('Error Message', 'Unknown error')}")
        return None
    
    daily_prices = data["Time Series (Daily)"]
    df = pd.DataFrame.from_dict(daily_prices, orient="index")
    df = df.reset_index().rename(columns={"index": "date"})
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")
    
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    df = df[["date", "close"]]  
    df["close"] = df["close"].astype(float)
    
    return df

api_key = "YA9LEXMLC8NH72G1"
tickers = ["TKO", "EDR"]  


stock_data = {}
for ticker in tickers:
    print(f"Fetching data for {ticker}...")
    df = fetch_stock_data(ticker, api_key)
    if df is not None:
        stock_data[ticker] = df.assign(ticker=ticker)


if stock_data:
    combined_data = pd.concat(stock_data.values(), ignore_index=True)
    
    
    five_years_ago = datetime.now() - pd.DateOffset(years=5)
    combined_data = combined_data[combined_data["date"] >= five_years_ago]
    
    fig = px.line(
        combined_data,
        x="date",
        y="close",
        color="ticker",
        title="5-Year Stock Price Trends",
        labels={"close": "Closing Price", "date": "Date"},
    )
    fig.update_layout(xaxis_title="Date", yaxis_title="Closing Price", legend_title="Ticker")
    fig.show()
else:
    print("No valid data fetched. Please check your API key or tickers.")

#In this plot, we can see that the stock prices for both TKO and EDR have been increasing over the past 5 years, but TKO has taken a significant jump while EDR is barely rising. On the AlphaVantage API website, I was not able to pull any WWE data.

## Step 3

Just like every other nerdy hobby, professional wrestling draws dedicated fans. Wrestling fans often go to cagematch.net to leave reviews for matches, shows, and wrestlers. The following link contains the top 100 matches on cagematch: https://www.cagematch.net/?id=111&view=statistics

* What is the correlation between WON ratings and cagematch ratings?

** Which wrestler has the most matches in the top 100?

*** Which promotion has the most matches in the top 100? 

**** What is each promotion's average WON rating?

***** Select any single match and get the comments and ratings for that match into a data frame.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page containing the table
url = "https://www.cagematch.net/?id=111&view=statistics"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Locate the div with class "Table"
    table_div = soup.find("div", {"class": "Table"})
    
    if table_div:
        # Locate all rows within the table
        rows = table_div.find_all("tr")[1:]  # Skip the header row
        
        # Extract data from each row
        data = []
        for row in rows:
            columns = row.find_all("td")
            data.append({
                "Rank": columns[0].text.strip(),
                "Date": columns[1].text.strip(),
                "Promotion": columns[2].img["alt"].strip() if columns[2].img else None,
                "Match": columns[3].text.strip(),
                "WON Rating": columns[4].text.strip(),
                "Match Type": columns[5].text.strip(),
                "Rating": columns[6].text.strip(),
                "Votes": columns[7].text.strip()
            })
        
        # Convert to a DataFrame
        df = pd.DataFrame(data)
        
        # Display or save the DataFrame
        print(df)

## Question 1

In [ ]:
#* What is the correlation between WON ratings and cagematch ratings?
import re

def convert_won_rating(rating):
    if not isinstance(rating, str):  
        return None
    match = re.match(r"\*{1,5}(?:\*\/\d|\*\d\/\d)?", rating)  
    if match:
        base = rating.count("*")  
        fraction = 0.25 if "1/4" in rating else 0.5 if "1/2" in rating else 0.75 if "3/4" in rating else 0
        return base + fraction
    return None  

df["WON Rating Numeric"] = df["WON Rating"].apply(convert_won_rating)

df_cleaned = df.dropna(subset=["WON Rating Numeric", "Rating"])

correlation = df_cleaned["WON Rating Numeric"].corr(df_cleaned["Rating"])
print(f"Correlation between WON Ratings and Cagematch Ratings: {correlation}")

## Question 2

In [ ]:
#** Which wrestler has the most matches in the top 100?
from collections import Counter

all_wrestlers = []

# Process each match in the dataset
for match in df["Match"]:
    # Split the match into sides using " vs. "
    sides = match.split(" vs. ") if " vs. " in match else [match]
    
    # For each side, split further by " & " to account for tag teams
    for side in sides:
        wrestlers = side.split(" & ")
        all_wrestlers.extend(wrestlers)

# Count the occurrences of each wrestler
wrestler_counts = Counter(all_wrestlers)

# Find the wrestler with the most matches
most_common_wrestler = wrestler_counts.most_common(1)[0]
print(f"Wrestler with the most matches: {most_common_wrestler[0]} ({most_common_wrestler[1]} matches)")

## Question 3

In [ ]:
# Count occurrences of each promotion
promotion_counts = df["Promotion"].value_counts()

# Top promotion
top_promotion = promotion_counts.idxmax()
top_promotion_count = promotion_counts.max()

print(f"Top promotion: {top_promotion} with {top_promotion_count} matches")

## Question 4

In [ ]:
#**** What is each promotion's average WON rating?
import re

def convert_won_rating(rating):
    if not isinstance(rating, str):  
        return None
    match = re.match(r"\*{1,5}(?:\*\/\d|\*\d\/\d)?", rating)  
    if match:
        base = rating.count("*")  
        fraction = 0.25 if "1/4" in rating else 0.5 if "1/2" in rating else 0.75 if "3/4" in rating else 0
        return base + fraction
    return None  

df["WON Rating"] = df["WON Rating"].apply(convert_won_rating)

print(df["WON Rating"].isna().sum(), "NaN values remain in WON Rating.")

avg_won_ratings = df.groupby("Promotion")["WON Rating"].mean()

print("Average WON Ratings by Promotion:")
print(avg_won_ratings)

## Question 5

In [ ]:
##Select any single match and get the comments and ratings for that match into a data frame.
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Send a GET request to the webpage
url = "https://www.cagematch.net/?id=111&nr=8034&page=99"
response = requests.get(url)

# Step 2: Parse the page with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Step 3: Find the comment and rating section
comments_section = soup.find_all("div", class_="Comment")

# Step 4: Initialize a list to store comment data
comments_data = []

# Step 5: Extract the commenter name, comment content, and any rating
for comment in comments_section:
    commenter = comment.find("a")  # The name of the commenter is in an <a> tag
    comment_text = comment.find("div", class_="CommentContents")  # The actual comment
    if commenter and comment_text:
        commenter_name = commenter.get_text(strip=True)
        comment_content = comment_text.get_text(strip=True)
        
        # Optional: Check if the comment has a rating number
        rating = comment.find("span", class_="Rating")  # Adjust this based on page structure
        if rating:
            rating_value = rating.get_text(strip=True)
        else:
            rating_value = "No rating"

        # Append the data as a dictionary to the list
        comments_data.append({
            "Commenter": commenter_name,
            "Rating": rating_value,
            "Comment": comment_content
        })

# Step 6: Convert the list of dictionaries into a pandas DataFrame
df_comments = pd.DataFrame(comments_data)

# Step 7: Display the DataFrame
print(df_comments)

## Step 4

You can't have matches without wrestlers. The following link contains the top 100 wrestlers, according to cagematch: https://www.cagematch.net/?id=2&view=statistics

*** Of the top 100, who has wrestled the most matches?

***** Of the top 100, which wrestler has the best win/loss?


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_wrestler_links():
    """
    Scrapes the top 100 wrestlers' names and profile links from the main page.
    
    Returns:
        pd.DataFrame: A DataFrame containing wrestler names and profile links.
    """
    url = "https://www.cagematch.net/?id=2&view=statistics"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to fetch the main page.")
        return pd.DataFrame()

    # Parse the page
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="TBase TableBorderColor")  # Updated class name
    if not table:
        print("Table not found!")
        return pd.DataFrame()

    rows = table.find_all("tr", class_=["TRow1", "TRow2"])  # Rows with data

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:  # Ensure the row contains data
            name_tag = cols[1].find("a")  # Link to wrestler's profile
            if name_tag:
                name = name_tag.text.strip()
                link = "https://www.cagematch.net/" + name_tag["href"]
                data.append({"Wrestler": name, "Profile Link": link})

    return pd.DataFrame(data)

# Scrape wrestler links
wrestler_links_df = scrape_wrestler_links()
print(wrestler_links_df.head())

In [ ]:
#*** Of the top 100, who has wrestled the most matches?
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode

# Base URL for the website
base_url = "https://www.cagematch.net/"

# URL to scrape the list of wrestlers
url = "https://www.cagematch.net/?id=2&view=statistics"

# Dictionary to store wrestler names and their total matches
wrestler_matches = []

# Step 1: Request the page with the list
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Extract all wrestler URLs (hrefs) from the 'Gimmick' column
gimmick_links = soup.find_all('a', href=True)
wrestler_urls = [base_url + link['href'].replace("&amp;", "&") for link in gimmick_links if 'gimmick=' in link['href']]

# Step 3: For each wrestler URL, modify the URL and scrape the total matches
for wrestler_url in wrestler_urls:
    # Parse the URL and remove the gimmick and any other query parameters
    parsed_url = urlparse(wrestler_url)
    query_params = parse_qs(parsed_url.query)

    # Remove the gimmick parameter
    query_params.pop('gimmick', None)

    # Rebuild the URL without the gimmick parameter and append &page=22
    new_query = urlencode(query_params, doseq=True)
    wrestler_page_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{new_query}&page=22"

    # Send the request to the new page URL
    wrestler_response = requests.get(wrestler_page_url)
    wrestler_soup = BeautifulSoup(wrestler_response.text, 'html.parser')
    
    # Attempt to find the Total Matches in the div with class 'InformationBoxContents'
    total_matches_section = wrestler_soup.find('div', class_='InformationBoxContents')
    
    if total_matches_section:
        # Extract the number (strip whitespace and clean it up)
        total_matches = int(total_matches_section.text.strip().split()[0])  # Convert to integer for sorting
        wrestler_name = wrestler_url.split('=')[-1]  # Extract wrestler name from the URL (after 'gimmick=')
        wrestler_matches.append((wrestler_name, total_matches))
    else:
        print(f"Wrestler: {wrestler_url} - Total Matches not found")

# Step 4: Sort the wrestlers by total matches in descending order
sorted_wrestlers = sorted(wrestler_matches, key=lambda x: x[1], reverse=True)

# Step 5: Print the sorted wrestlers
print("Wrestlers sorted by Total Matches:")
for wrestler, matches in sorted_wrestlers:
    print(f"{wrestler}: {matches} matches")


### Answer is Ric Flair

In [ ]:
##***** Of the top 100, which wrestler has the best win/loss?

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urlencode

# Base URL for the website
base_url = "https://www.cagematch.net/"

# URL to scrape the list of wrestlers
url = "https://www.cagematch.net/?id=2&view=statistics"

# List to store wrestler names and their win percentage
wrestler_win_percentages = []

# Step 1: Request the page with the list
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Print the page HTML to check if the structure is correct
print(soup.prettify())  # Prints the HTML of the page in a readable format

# Step 2: Extract all wrestler URLs (hrefs) from the 'Gimmick' column
gimmick_links = soup.find_all('a', href=True)
wrestler_urls = [base_url + link['href'].replace("&amp;", "&") for link in gimmick_links if 'gimmick=' in link['href']]

# Debugging: Print all extracted wrestler URLs to ensure we are collecting them correctly
print("Wrestler URLs extracted:")
for url in wrestler_urls:
    print(url)

# Step 3: For each wrestler URL, modify the URL and scrape the win percentage
for wrestler_url in wrestler_urls:
    # Parse the URL and remove the gimmick and any other query parameters
    parsed_url = urlparse(wrestler_url)
    query_params = parse_qs(parsed_url.query)

    # Remove the gimmick parameter
    query_params.pop('gimmick', None)

    # Rebuild the URL without the gimmick parameter and append &page=22
    new_query = urlencode(query_params, doseq=True)
    wrestler_page_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{new_query}&page=22"

    # Send the request to the new page URL
    wrestler_response = requests.get(wrestler_page_url)
    wrestler_soup = BeautifulSoup(wrestler_response.text, 'html.parser')

    # Debugging: Print the wrestler page HTML to see the structure
    # print(f"HTML for {wrestler_url}:")
    # print(wrestler_soup.prettify())  # Uncomment this if needed for debugging

    # Attempt to find the win percentage in the div with class 'InformationBoxContents'
    win_percentage_section = wrestler_soup.find_all('div', class_='InformationBoxContents')

    # Debugging: Print all the found InformationBoxContents divs to see their content
    # print(f"Found InformationBoxContents divs for {wrestler_url}:")
    # for section in win_percentage_section:
    #     print(section.text.strip())

    # Find the div that contains the win percentage (format should be "1195 (49.7%)")
    for section in win_percentage_section:
        win_percentage_text = section.text.strip()

        # We are looking for the format that includes the percentage in parentheses
        if '(' in win_percentage_text and ')' in win_percentage_text:
            # Extract the percentage inside parentheses (e.g., "49.7%")
            start = win_percentage_text.find('(') + 1
            end = win_percentage_text.find(')')
            win_percentage_str = win_percentage_text[start:end].strip('%')  # Extract percentage without '%'

            # Debugging: Print the extracted win percentage
            print(f"Extracted win percentage for {wrestler_url}: {win_percentage_str}%")

            try:
                # Convert the win percentage to float and store it
                win_percentage = float(win_percentage_str)  # Convert to float
                wrestler_name = wrestler_url.split('=')[-1]  # Extract wrestler name from the URL
                wrestler_win_percentages.append((wrestler_name, win_percentage))
                break  # If we found a valid win percentage, break the loop
            except ValueError:
                print(f"Error converting win percentage for {wrestler_url}. Got: {win_percentage_str}")
        else:
            print(f"Win percentage format not found in div for wrestler: {wrestler_url}")

# Step 4: Sort the wrestlers by win percentage in descending order
sorted_wrestlers = sorted(wrestler_win_percentages, key=lambda x: x[1], reverse=True)

# Step 5: Print the sorted wrestlers
print("Wrestlers sorted by Win Percentage:")
for wrestler, win_percent in sorted_wrestlers:
    print(f"{wrestler}: {win_percent}% win rate")

### Answer is Gene Okerlund

## Step 5

With all of this work out of the way, we can start getting down to strategy.

First, what talent should WWE pursue? Advise carefully.

WWE should prioritize pursuing talent with high win/loss ratios, as these wrestlers demonstrate dominance and appeal to audiences. Wrestlers with high match counts and strong performance metrics bring experience and visibility, making them valuable additions. WWE could target stars like Kazuchika Okada or Will Ospreay from NJPW or AEW, who have a proven track record of success and cross-promotional appeal. Additionally, WWE should consider younger, underrated wrestlers with strong in-ring skills who can grow with the company and enhance its long-term roster.

Second, reconcile what you found in steps 3 and 4 with Netflix's relationship with WWE. Use the data from the following page to help make your case: https://wrestlenomics.com/tv-ratings/

Netflix’s partnership with WWE positions the platform to attract younger and diverse audiences, aligning with WWE’s digital expansion goals. For this relationship to thrive, WWE must ensure its content sustains strong engagement and viewership metrics. According to Wrestlenomics’ TV ratings data, WWE programs like Raw and SmackDown already dominate key demographics, such as the 18-49 age group. WWE should leverage this momentum by featuring its top-performing talent and creating compelling storylines to maintain viewership levels on Netflix.

Third, do you have any further recommendations for WWE?

To solidify its growth, WWE should continue expanding globally by collaborating with international wrestling promotions and recruiting talent from emerging markets. Strengthening its women’s division and tag team categories could diversify its offerings and draw new viewers. WWE should also focus on enhancing its storytelling and character development to deepen fan engagement. Finally, leveraging the Netflix platform, WWE can explore exclusive series or documentaries that spotlight its history, top stars, and behind-the-scenes action to attract broader audiences.